## Construct a Galaxy model using Plummer potential

We use the "N-body" units, in which G=1: length = 1 kpc, velocity = 1 km/s, mass = 232500 Msun

In [ ]:
import agama
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm, trange

use_self_consistent_model = False

In [ ]:
def create_self_consistent_model(potential, df, verbose=False, plot=False):
    print("Creating a self-consistent model")

    # initial guess for the density profile - deliberately a wrong one
    dens = agama.Density(type="Dehnen", mass=0.1, scaleRadius=0.5)

    # define the self-consistent model consisting of a single component
    params = dict(rminSph=0.001, rmaxSph=1000.0, sizeRadialSph=40, lmaxAngularSph=0)
    comp = agama.Component(df=df, density=dens, disklike=False, **params)
    scm = agama.SelfConsistentModel(**params)
    scm.components = [comp]

    # prepare visualization
    r = np.logspace(-2.0, 2.0)
    xyz = np.vstack((r, r * 0, r * 0)).T
    if plot:
        plt.plot(r, dens.density(xyz), label="Init density")
        plt.plot(r, potential.density(xyz), label="True density", c="k")[0].set_dashes(
            [4, 4]
        )

    # perform several iterations of self-consistent modelling procedure
    for i in range(10):
        scm.iterate()
        if verbose:
            print(
                "Iteration %i, Phi(0)=%g, Mass=%g"
                % (i, scm.potential.potential(0, 0, 0), scm.potential.totalMass())
            )
        if plot:
            plt.plot(r, scm.potential.density(xyz), label="Iteration #" + str(i))

    # save the final density/potential profile
    scm.potential.export("simple_scm.ini")

    if plot:
        # show the results
        plt.legend(loc="lower left")
        plt.xlabel("r")
        plt.ylabel(r"$\rho$")
        plt.xscale("log")
        plt.yscale("log")
        plt.ylim(1e-5, 1)
        plt.xlim(0.02, 20)
        plt.show()

    return scm

In [ ]:
N = 10_000
potential = agama.Potential(type="Plummer", scaleRadius=0.01, mass=1e6 / 232500)
df = agama.DistributionFunction(type="QuasiSpherical", potential=potential)

if use_self_consistent_model:
    # create and write out an N−body realization of the model
    scm = create_self_consistent_model(
        potential=potential,
        df=df,
        plot=True,
        verbose=False,
    )

    c_pot = scm.potential
    snap = agama.GalaxyModel(scm.potential, df).sample(N)
    agama.writeSnapshot("plummer.dat", snap)
else:
    c_df = agama.DistributionFunction(type="quasispherical", potential=potential)
    c_gm = agama.GalaxyModel(potential, df)

    c_pot = potential
    snap = c_gm.sample(N)

## Perform an N-body simulation

In [ ]:
# create an isolated star cluster
xv, m = snap[0], snap[1]

# shift it to some initial point in the Galaxy
c_center = np.hstack([2.0, 0, 0, 0, -100, 50])
xv += c_center

potfile = "MWPotentialHunter24_rotating.ini"
try:
    g_pot = agama.Potential(potfile)
except RuntimeError:
    print(
        "You need to create the barred Milky Way potential by running 'python Agama/py/example_mw_potential_hunter24.py'"
    )
    exit(1)

In [ ]:
%%capture
%matplotlib inline
import matplotlib.animation

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 200

# Initialize plot and simulation parameters
fig = plt.figure(figsize=(10, 10), dpi=75)
ax = plt.axes([0.08, 0.08, 0.9, 0.9])

sim_time = 0.25 * 10  # total simulation time in units of kpc/(km/s) = 0.98 Gyr
num_intervals = 32 * 10
num_subint = 16
interval = sim_time / num_intervals


# Update plot at each iteration `i`
def update(i, ax, nbody_data, N, g_pot):
    snap, c_bound, orbit_center, time = nbody_data[i]

    ax.cla()
    ax.scatter(
        snap[:, 0],
        snap[:, 1],
        c=c_bound,
        cmap="bwr_r",
        vmin=0,
        vmax=1,
        s=2,
        linewidths=0,
    )
    ax.plot(
        orbit_center[0 : i * num_subint, 0],
        orbit_center[0 : i * num_subint, 1],
        color="k",
    )
    ax.text(
        0.01,
        0.99,
        "time=%.4f, bound fraction=%.3f" % (time, np.sum(c_bound) * 1.0 / N),
        ha="left",
        va="top",
        transform=ax.transAxes,
    )

    # Draw Milky way potential
    grid = np.linspace(-2.5, 2.5, 101)
    xyz = np.column_stack(
        [
            np.repeat(grid, len(grid)),
            np.tile(grid, len(grid)),
            np.zeros(len(grid) ** 2),
        ]
    )
    den = g_pot.density(xyz, t=time).reshape(len(grid), len(grid)).T
    ax.contour(
        grid, grid, np.log10(den), levels=np.linspace(3, 7, 17), cmap="cool", zorder=2
    )
    ax.set_xlim(min(grid), max(grid))
    ax.set_ylim(min(grid), max(grid))

In [ ]:
# Initialize data for simulation

# calculate how cluster's center will move in a Milky Way potentia
time_center, orbit_center = agama.orbit(
    potential=g_pot,
    ic=c_center,
    time=sim_time,
    trajsize=num_intervals * num_subint + 1,
)
cpot = c_pot
snap = xv.copy()

# Create a variable for simulation artifacts
nbody_data = {}

print("Start simulation...")

for i in trange(num_intervals + 1):  # Simulate `num_intervals` frames
    # determine which particles remain bound to the satellite
    c_bound = (
        cpot.potential(
            snap[:, 0:3] - orbit_center[i * num_subint, 0:3]
        )  # error in original code?
        + 0.5 * np.sum((snap[:, 3:6] - orbit_center[i * num_subint, 3:6]) ** 2, axis=1)
        < 0
    )
    time = i * interval

    # Fill `nbody_data`
    nbody_data[i] = (snap, c_bound, orbit_center, time)

    if i == num_intervals:
        continue

    # evolve the cluster for some time:
    # initialize the time-dependent total potential (host + moving sat) on this time interval
    t_pot = agama.Potential(
        g_pot,
        agama.Potential(
            potential=cpot,
            center=np.column_stack((time_center, orbit_center)),
        ),
    )

    # compute the trajectories of all particles moving in the combined potential of the host galaxy and the moving satellite
    snap = np.vstack(
        agama.orbit(
            ic=snap,
            potential=t_pot,
            time=interval,
            timestart=time,
            trajsize=1,
            accuracy=1e-5,
            verbose=False,
        )[:, 1]
    )
    # update the potential of the satellite (using a spherical monopole approximation)
    cpot = agama.Potential(
        type="multipole",
        particles=(snap[:, 0:3] - orbit_center[(i + 1) * num_subint, 0:3], m),
        symmetry="s",
    )

In [ ]:
from functools import partial

ani = matplotlib.animation.FuncAnimation(
    fig=fig,
    func=partial(
        update,
        ax=ax,
        nbody_data=nbody_data,
        N=N,
        g_pot=g_pot,
    ),
    frames=num_intervals + 1,
    interval=250,
    cache_frame_data=False,
)

ani.save("sim.gif", writer="pillow")

In [ ]:
ani